In [1]:
%%capture
#!pip install wfdb
BASEPATH = '../'
import sys
sys.path.insert(0,'../py')
print(sys.path)
import numpy as np
import os

from utils import reset_seed, plot_spectogram, plot_loss, load_data
from utils import integer_mapping, mapping_AAMI, MAP_AAMI, INT_MAP_DICT
from data_handling import DataHandling
from data_preprocessing import DataPreprocessing
from report import Reports

reset_seed()

In [2]:
#Load Preprocessed data
x_train, y_train = load_data(file_path = '../data/DS1_features_train.dat')
x_train.shape, len(y_train)
#Load Preprocessed data
x_val, y_val = load_data(file_path = '../data/DS1_features_val.dat')
x_val.shape, len(y_val)

File loaded: ../data/DS1_features_train.dat
File loaded: ../data/DS1_features_val.dat


((29152, 7, 33), 29152)

In [3]:
dh = DataHandling()
rp1 = dh.report_stats_table([y_train, y_val], ['train','val'])
rp1

,N,L,R,j,e,V,E,A,S,a,J,F,f,/,Q
train,5000,5526,5292,5005,5005,5156,5032,5103,5001,5040,5016,5220,0,0,5004
val,2200,2368,2268,2205,2205,2210,2201,2430,2201,2220,2210,2232,0,0,2202


In [4]:
#binary N vs. V
from data_augmentation import slice_data

x_train_sliced, y_train_sliced = slice_data(x_train,y_train,['N','V'])

dh = DataHandling()
rp1 = dh.report_stats_table([y_train_sliced], ['y_train_sliced'])
rp1

10156


,N,L,R,j,e,V,E,A,S,a,J,F,f,/,Q
y_train_sliced,5000,0,0,0,0,5156,0,0,0,0,0,0,0,0,0


In [5]:
x_train = x_train_sliced
y_train = y_train_sliced

In [6]:
y_train = integer_mapping(y_train, INT_MAP_DICT)
y_val = integer_mapping(y_val, INT_MAP_DICT)

In [7]:
x_train = x_train.reshape(x_train.shape[0],-1)
x_val = x_val.reshape(x_val.shape[0],-1)

In [8]:
x_train.shape, x_val.shape

((10156, 231), (29152, 231))

In [9]:
%%time
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovr',kernel='rbf',C=0.9)
clf.fit(x_train, y_train)

CPU times: user 3.97 s, sys: 63.8 ms, total: 4.03 s
Wall time: 4.03 s


SVC(C=0.9)

# Test

In [10]:
#Load Preprocessed Test Set
features_test, y_test = load_data(file_path = '../data/DS2_features.dat')
x_test = features_test
y_true = y_test.copy()
features_test.shape, len(y_test)

x_test = x_test.reshape(x_test.shape[0],-1)
x_test.shape

File loaded: ../data/DS2_features.dat


(49693, 231)

In [11]:
%%time
y_pred = clf.predict(x_test)

CPU times: user 20 s, sys: 717 µs, total: 20 s
Wall time: 20 s


In [12]:
y_pred = integer_mapping(y_pred, INT_MAP_DICT, inverse=True)

In [13]:
dh = DataHandling()
rp2 = dh.report_stats_table([y_pred, y_true], ['y_pred','y_true'])
rp2

,N,L,R,j,e,V,E,A,S,a,J,F,f,/,Q
y_pred,42344,0,0,0,0,7349,0,0,0,0,0,0,0,0,0
y_true,36428,4124,3476,213,0,3219,1,1736,0,50,51,388,0,0,7


In [14]:
labels = [k for k,v in INT_MAP_DICT.items()]
report = Reports(y_true=y_true, y_pred=y_pred, labels=labels)
normalize = None
#report.plot_confusion_matrix(normalize=normalize, values_format='')
print(report.the_classification_report())
#report.metrics_table()


y_pred_AAMI = mapping_AAMI(y_pred, MAP_AAMI)
y_true_AAMI = mapping_AAMI(y_true, MAP_AAMI)
labels_AAMI = ['N','V','S','F','Q']
report = Reports(y_true=y_true_AAMI, y_pred=y_pred_AAMI, labels=labels_AAMI)
#report.plot_confusion_matrix(normalize=normalize, values_format='')
print(report.the_classification_report())
report.metrics_table()

/home/mypc/anaconda3/envs/ecg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mypc/anaconda3/envs/ecg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mypc/anaconda3/envs/ecg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           A     0.0000    0.0000    0.0000      1736
           E     0.0000    0.0000    0.0000         1
           F     0.0000    0.0000    0.0000       388
           J     0.0000    0.0000    0.0000        51
           L     0.0000    0.0000    0.0000      4124
           N     0.8290    0.9636    0.8913     36428
           Q     0.0000    0.0000    0.0000         7
           R     0.0000    0.0000    0.0000      3476
           V     0.4006    0.9146    0.5572      3219
           a     0.0000    0.0000    0.0000        50
           j     0.0000    0.0000    0.0000       213

    accuracy                         0.7656     49693
   macro avg     0.1118    0.1707    0.1317     49693
weighted avg     0.6337    0.7656    0.6894     49693



/home/mypc/anaconda3/envs/ecg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mypc/anaconda3/envs/ecg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mypc/anaconda3/envs/ecg/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           F     0.0000    0.0000    0.0000       388
           N     0.9478    0.9072    0.9270     44241
           Q     0.0000    0.0000    0.0000         7
           S     0.0000    0.0000    0.0000      1837
           V     0.4006    0.9143    0.5571      3220

    accuracy                         0.8669     49693
   macro avg     0.2697    0.3643    0.2968     49693
weighted avg     0.8698    0.8669    0.8614     49693



/media/mypc/STORAGE1/codes/heartbeat/ipynb/../py/report.py:54: RuntimeWarning: invalid value encountered in true_divide
  PPV = TP/(TP+FP)*100 # Precision, positive predictive value (PPV)


,Class,(PPV)Precision,(Sensitivity)Recall,Specificity,Accuracy
0,F,NaN,0.00,100.00,99.22
1,N,94.78,90.72,59.46,87.29
2,Q,NaN,0.00,100.00,99.99
3,S,NaN,0.00,100.00,96.30
4,V,40.06,91.43,90.52,90.58
